In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import time
from IPython.display import clear_output
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint as MC
from tensorflow.keras import backend as K
from sklearn.metrics import confusion_matrix, roc_curve, auc, recall_score, accuracy_score, balanced_accuracy_score, precision_score
import pickle
from IPython.display import FileLink



root = '/kaggle/input/rsna-str-pulmonary-embolism-detection'
for item in os.listdir(root):
    path = os.path.join(root, item)
    if os.path.isfile(path):
        print(path)

# Load Data

Next, we load all '.csv' files into memory and peek into their makeup.

In [ ]:
print('Reading train data...')
data = pd.read_csv("../input/rsna-str-pulmonary-embolism-detection/train.csv")
print(data.shape)
data.head()

# Negative exam selection

Only the positive exams containing pulmonary embolism have information on rv_lv_ratio_gte_1 and rv_lv_ratio_lt_1. So next we select only the images from the positive exams.

In [ ]:
#Selecting only positive exams
data = data.loc[data['negative_exam_for_pe'] == 0] 

# Agrupar por col_groupby 
col_index = 'SOPInstanceUID'
col_groupby = 'StudyInstanceUID'
data = data[data[col_groupby].duplicated()==False].reset_index(drop=True)
data

In [ ]:
#For testing a smaller dataset
data_small = data[:250]
data_small.shape

# Train/Test Split

Since this project is for academic purposes, we will split the given train set into train+test set, to evaluate performance of the model better. Since the data is already structured by exam, a simple train_test_split(80% train, 20% test) is employed.

In [ ]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(data, test_size=0.2, random_state=42, shuffle=False)
train

# Data pre-processing functions

- normalize_image: Converts the CT scan data from Hounsfield scale to a 0-1 scale.
- resize_volume: Resizes the volume of each exam to lower computational costs and ensure training uniformity. We resized to 128x128 on the slice level and 64 on exam depth (number of slices).

These functions were adapted from '3D image classification from CT scans' example in the Keras package documentation

In [ ]:
def normalize_image(image):
    min = -1000
    max = 400
    image[image < min] = min
    image[image > max] = max
    image = (image - min) / (max - min)
    image = image.astype("float32")
    return image

In [ ]:
from scipy import ndimage
def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    #img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

# Exam reader

The function get_exam gets the image arrays from a Dicom image and groups them by exam. This function was adapted from a function developed by [eladwar](https://www.kaggle.com/eladwar) in this notebook [here](https://www.kaggle.com/eladwar/20-seconds-or-less#VTK-is-mostly-written-in-C++-making-it-incredibly-efficient.-By-using-this-library-you-can-save-loads-of-memory-and-time.).

In [ ]:
import vtk
from vtk.util import numpy_support
import cv2

def get_exam(path):
    x = 0
    #scan=np.array([])
    n_slices = len([name for name in os.listdir(directory)])  #number of slices in exam = number of images in exam file
    exam = np.zeros((512, 512, n_slices))
    for file in os.listdir(directory):
        f = os.path.join(directory, file)

        reader = vtk.vtkDICOMImageReader()
        reader.SetFileName(f)
        reader.Update()
        _extent = reader.GetDataExtent()
        ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

        ConstPixelSpacing = reader.GetPixelSpacing()
        imageData = reader.GetOutput()
        pointData = imageData.GetPointData()
        arrayData = pointData.GetArray(0)
        ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
        ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')
        ArrayDicom = cv2.resize(ArrayDicom,(512,512))
        #ArrayDicom = ArrayDicom.reshape((1,ArrayDicom.shape[0],ArrayDicom.shape[1]))  #For np.concatenate method

        ArrayDicom = normalize_image(ArrayDicom)


        exam[:, :, x] = ArrayDicom  #Simply allocating each slice to the vector is more efficient than concatenating

        #if scan.size==0:
            #scan = ArrayDicom
        #else:
            #scan = np.concatenate((scan, ArrayDicom))
        x+=1
    exam[:, :, 1].shape
    exam = resize_volume(exam)
    
    exam = exam.reshape((exam.shape[0],exam.shape[1],exam.shape[2],1))
    
    return exam


#Check exam shape (width, height, depth)
directory = os.fsencode("../input/rsna-str-pulmonary-embolism-detection/train/017320409cc6/fbd7d4bb6cb5")
exam = get_exam(directory)
exam.shape

In [ ]:
#plot a slice from the example exam
import matplotlib.pyplot as plt

plt.imshow(np.squeeze(exam[:, :, 20]), cmap="gray")

# Model creation

A 3-dimensional Convolution Neural Network is proposed, consisting of several modules of 3D conv, maxpool and batch normalization layers. The model was adapted from this [paper](https://arxiv.org/pdf/2007.13224.pdf) by Zunair et al.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers

#initializer = tf.keras.initializers.RandomNormal(mean=3., stddev=1.)

inputs = keras.Input((128, 128, 64, 1))

x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPool3D(pool_size=2)(x)
x = layers.BatchNormalization()(x)

x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPool3D(pool_size=2)(x)
x = layers.BatchNormalization()(x)

x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPool3D(pool_size=2)(x)
x = layers.BatchNormalization()(x)

x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPool3D(pool_size=2)(x)
x = layers.BatchNormalization()(x)

x = layers.GlobalAveragePooling3D()(x)
x = layers.Dense(units=512, activation="relu")(x)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(units=1, activation="sigmoid")(x)

# Define the model.
model = keras.Model(inputs=inputs, outputs=outputs, name="3dcnn")

#initial_learning_rate = 0.0001
#lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
#)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    #metrics=["acc"],
    metrics=tf.keras.metrics.AUC()
)
model.summary()
model.save('3d_CNN_rvlv.h5')
del model
K.clear_session()
gc.collect()

# Training Model

custom_dcom_image_generator is a function that constructs the batch on which we will train and test. If train, it will yield (return a generator, iterable only once) with the X and Y, where X is a group of images If test, it will yield the X on which we will predict.

In [ ]:
def custom_dcom_image_generator(batch_size, dataset, test=False, debug=False):
    
    fnames = dataset[['StudyInstanceUID', 'SeriesInstanceUID']]
    
    if not test:
        Y = dataset[['rv_lv_ratio_lt_1']]
        prefix = 'input/rsna-str-pulmonary-embolism-detection/train'
        
    else:
        prefix = 'input/rsna-str-pulmonary-embolism-detection/train'
    
    X = np.array([])
    batch = 0
    for st, sr in fnames.values:
        if debug:
            print(f"Current file: ../{prefix}/{st}/{sr}")

        temp = get_exam(f"../{prefix}/{st}/{sr}")
        temp = temp.reshape((1, temp.shape[0], temp.shape[1], temp.shape[2], temp.shape[3]))
        
        if X.size==0:
            X = temp
        else:
            X = np.concatenate((X, temp))
            
        
        del st, sr
        
        #If we reached the end of the batch
        if len(X) == batch_size:
            if test:
                #yield is used to save memory
                yield X
                del X
            else:
                yield X, Y[batch*batch_size:(batch+1)*batch_size].values
                del X
                
            gc.collect()
            X = np.array([])
            batch += 1
        
    if test:
        yield X
    else:
        yield X, Y[batch*batch_size:(batch+1)*batch_size].values
        del Y
    del X
    gc.collect()
    return

In [ ]:
def train_model(model_path, train_data, train_size, batch_size, max_train_time, debug):
    #Train loop
    for n, (x, y) in enumerate(custom_dcom_image_generator(batch_size, train_data, False, debug)):

        if len(x) < batch_size:#Tries to filter out empty or short data
            model = load_model(model_path)
            break
        
        clear_output(wait=True)
        print("Training batch: %i - %i" %(batch_size*n, batch_size*(n+1)))
        model = load_model(model_path)
        #print("check 1")
        hist = model.fit(
            x[:train_size], 
            y[:train_size],

            callbacks = checkpoint,

            #validation_split=0.2,
            epochs=50,
            #batch_size=2,
            verbose=debug
        )
        print(hist)
        
        try:
            history = np.concatenate(history, hist.history)
        except:
            history = hist.history
        #print(history)
            

        print("Metrics for batch validation:")
        model.evaluate(x[train_size:],
                       y[train_size:]
                      )

        #To make sure that our model doesn't train overtime
        if time.time() - start >= max_train_time:
            print("Time's up!")
            break

        model.save('3d_CNN_rvlv.h5')
        del model, x, y, hist
        #del x, y, hist
        K.clear_session()
        gc.collect()
    
    
    return history, model
    #return model

In [ ]:
history= np.array([])
start = time.time()
train_data=train.sample(frac=1)
batch_size = 10
debug = 0

train_size = int(batch_size*0.1)

max_train_time = 3600 * 4 #hours to seconds of training

checkpoint = MC(filepath='../working/3d_CNN_rvlv.h5', monitor='val_loss', save_best_only=True, verbose=1)
#Train loop
#history,trained_model = train_model('../working/3d_CNN_rvlv.h5', train_data, train_size, batch_size, max_train_time, debug)
history, trained_model = train_model('../working/3d_CNN_rvlv.h5', train_data, train_size, batch_size, max_train_time, debug)
trained_model.save('3d_CNN_rvlv_trained.h5')

In [ ]:
history

In [ ]:
predictions = {}
stopper = 3600 * 4 #4 hours limit for prediction
pred_start_time = time.time()\

p, c = time.time(), time.time()
batch_size = 1
l = 0
n = test.shape[0]

for x in custom_dcom_image_generator(batch_size, test, True, False):
    clear_output(wait=True)
    #model = load_model("../input/modelss/3d_CNN_rvlv_trained.h5")
    model = load_model("../working/3d_CNN_rvlv.h5")
    print(x.shape)
    preds = model.predict(x, verbose=1)
    
    try:
        print(preds)
        predictions += preds.tolist()
    
    except Exception as e:
        print(e)
        predictions = preds.tolist()
            
            
    l = (l+batch_size)%n
    p, c = c, time.time()
    print("One batch time: %.2f seconds" %(c-p))
    print("ETA: %.2f" %((n-l)*(c-p)/batch_size))
    
    if c - pred_start_time >= stopper:
        print("Time's up!")
        break
    
    del model
    K.clear_session()
    
    del x, preds
    gc.collect()

In [ ]:
predictions